<a href="https://colab.research.google.com/github/AkashNavgire02/Pneumonia-detection/blob/main/Pneumonia_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"akashnavgire","key":"2257b6cc8152bc2d200c88f9375329b5"}'}

In [ ]:
#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:25<00:00, 118MB/s]
100% 2.29G/2.29G [00:25<00:00, 95.1MB/s]


In [ ]:
!mkdir Xray_pneumonia

In [ ]:
!unzip /content/chest-xray-pneumonia.zip -d Xray_pneumonia

In [ ]:
train_dir = "/content/Xray_pneumonia/chest_xray/train"
test_dir = "/content/Xray_pneumonia/chest_xray/test" 

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150)) 

Found 5216 images belonging to 2 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150)) 

Found 5216 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (4,4), activation='selu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='selu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (2,2), activation='selu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(), 
    # 512 neuron hidden layer
    tf.keras.layers.Dense(9000, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(6000, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(3000, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1000, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(500, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(500, activation='selu'), 
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(200, activation='selu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(100, activation='selu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(50, activation='selu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='selu'),
    tf.keras.layers.Dropout(0.2),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('Organic') 
    # and 1 for the other ('Recycled')
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

model.summary()

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, 
                        verbose=1, mode='auto',restore_best_weights=True)
model.compile(optimizer=RMSprop(lr=0.001),
              loss='binary_crossentropy',
              metrics = ['acc'])

In [ ]:
history = model.fit_generator(train_generator,
                              validation_data=test_generator,
                              steps_per_epoch=250,epochs=20,
                              validation_steps=10,
                              verbose=2, callbacks=[monitor])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
250/250 - 690s - loss: 6.2061 - acc: 0.6165 - val_loss: 0.8345 - val_acc: 0.2350
Epoch 2/20
250/250 - 709s - loss: 46.3567 - acc: 0.6345 - val_loss: 0.5751 - val_acc: 0.7500
Epoch 3/20
250/250 - 698s - loss: 1.0142 - acc: 0.6725 - val_loss: 0.5672 - val_acc: 0.7850
Epoch 4/20
250/250 - 700s - loss: 23.3887 - acc: 0.6896 - val_loss: 0.5977 - val_acc: 0.7300
Epoch 5/20
250/250 - 697s - loss: 22.0521 - acc: 0.7056 - val_loss: 0.4767 - val_acc: 0.6550
Epoch 6/20
250/250 - 696s - loss: 0.6922 - acc: 0.7182 - val_loss: 0.5767 - val_acc: 0.7250
Epoch 7/20
250/250 - 695s - loss: 3.4530 - acc: 0.7342 - val_loss: 0.4156 - val_acc: 0.6750
Epoch 8/20
250/250 - 695s - loss: 0.9663 - acc: 0.8090 - val_loss: 0.1812 - val_acc: 0.9400
Epoch 9/20
250/250 - 696s - loss: 44.4504 - acc: 0.8943 - val_loss: 0.2478 - val_acc: 0.9200
Epoch 10/20
250/250 - 694s - loss: 1.2492 - acc: 0.8733 - val_loss: 0.1614 - val_acc: 0.9400

In [ ]:
test2_datagen  = ImageDataGenerator( rescale = 1.0/255. )
test_dir = "/content/Xray_pneumonia/chest_xray/val"
test_generator =  test2_datagen.flow_from_directory(test_dir,
                                                    batch_size=3,
                                                    class_mode  = 'input',
                                                    target_size = (150, 150),
                                                    shuffle=False)

Found 16 images belonging to 2 classes.


In [ ]:
y_prob = model.predict(test_generator)
y_prob


array([[0.39490113],
       [0.24873826],
       [0.1464289 ],
       [0.58173877],
       [0.90380037],
       [0.50134796],
       [0.19158189],
       [0.15281604],
       [1.        ],
       [1.        ],
       [1.        ],
       [0.8525218 ],
       [0.99225813],
       [1.        ],
       [0.8664411 ],
       [0.99969745]], dtype=float32)

In [ ]:
y_pred = ["Normal" if probs < 0.8 else "Pneumonia" for probs in y_prob]
y_pred

['Normal',
 'Normal',
 'Normal',
 'Normal',
 'Pneumonia',
 'Normal',
 'Normal',
 'Normal',
 'Pneumonia',
 'Pneumonia',
 'Pneumonia',
 'Pneumonia',
 'Pneumonia',
 'Pneumonia',
 'Pneumonia',
 'Pneumonia']

In [ ]:
import tensorflow as tf

from keras.models import load_model

model.save('model_Pneumonia.h5')

In [ ]:
/content/model_Pneumonia.h5
